# Calculate the Linderman score for the regression contribution analysis.

In [12]:
import pandas as pd
import numpy as np
import os
df = pd.read_csv("D:/Dropbox (Personal)/Personal Work/_Projects2023/01_city-never-was/_table/road_fatality_built_env_res=9_cluster=7.csv")[1:]
r1_df = df[df['Unnamed: 0']=="R-sq"]
r1_df.rename(columns = {
    
    "seg1":"demo",
    "seg2":"cluster",
    "seg3":"object",
    "seg12":"cluster_demo",
    "seg23":"cluster_object",
    "seg13":"demo_object",
    "seg123":"cluster_demo_object",
}, inplace = True)
# remove the columns if it contains "Unnamed"
r1_df = r1_df.loc[:, ~r1_df.columns.str.contains('^Unnamed')].astype(float)
r1_df

C:\Users\yuanzfan\AppData\Local\Temp\ipykernel_16760\3921181413.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r1_df.rename(columns = {


,demo,cluster,object,cluster_demo,cluster_object,demo_object,cluster_demo_object
16,0.3784,0.150823,0.159234,0.49051,0.254163,0.461015,0.54729


In [14]:
import itertools

r2_values = r1_df.to_dict('records')[0]
# Full model R²
r2_full = r2_values["cluster_demo_object"]

# Factors involved
factors = ["cluster", "demo", "object"]

# Compute LMG for each factor
lmg_scores = {factor: 0 for factor in factors}

# Generate all orderings of the factors
orderings = list(itertools.permutations(factors))

# Compute LMG contributions for each order
for ordering in orderings:
    prev_r2 = 0
    for i, factor in enumerate(ordering):
        included_factors = tuple(sorted(ordering[: i + 1]))
        r2_current = r2_values["_".join(included_factors)]
        contribution = r2_current - prev_r2
        lmg_scores[factor] += contribution
        prev_r2 = r2_current

# Average over all orderings
num_orderings = len(orderings)
lmg_scores = {factor: lmg / num_orderings for factor, lmg in lmg_scores.items()}

# Normalize to sum to 100%
lmg_scores = {factor: (lmg / r2_full) * 100 for factor, lmg in lmg_scores.items()}

# Convert to DataFrame for display
lmg_df = pd.DataFrame.from_dict(lmg_scores, orient="index", columns=["LMG Score (%)"])

# Display results
# tools.display_dataframe_to_user(name="LMG Scores", dataframe=lmg_df)
lmg_df

,LMG Score (%)
cluster,20.745694
demo,60.434839
object,18.819467
